In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
import scipy.optimize as so

In [2]:
df = pd.read_csv('data2.csv')

In [3]:
df.head(10)

,Date,SPTSX Index,TBBC1M Index,TBBC1M Index,SPTSXS Index,SPTSX60 Index,DJCASV Index,DJCASG Index,SPTSX Index.1,TBBC3M Index,Unnamed: 10
0,NaN,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS,PX_LAST,NaN,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS,DAY_TO_DAY_TOT_RETURN_GROSS_DVDS,EQY_DVD_YLD_12M,PX_LAST,NaN
1,NaN,Rm,Rf,Rf /12,Rs,Rb,Rh,Rl,D/P,TB,Rp
2,12/31/2009,2.9232,0.1400,0.0117,6.4581,2.1980,2.8745,1.9322,2.7465,0.1900,3.9700
3,1/29/2010,-5.3479,0.1300,0.0108,-2.9412,-6.2646,-4.2748,-7.0606,2.8833,0.1600,-3.3600
4,2/26/2010,4.9729,0.1700,0.0142,4.4461,5.0952,5.1154,4.3711,2.7360,0.1600,3.8000
5,3/31/2010,3.8074,0.2000,0.0167,3.4797,4.1065,5.9853,1.3966,2.6296,0.2800,4.1400
6,4/30/2010,1.6656,0.1900,0.0158,3.3702,1.5480,-0.5633,3.3570,2.5926,0.3900,4.1600
7,5/31/2010,-3.4783,0.2700,0.0225,-5.2111,-3.2370,-4.1439,-2.8400,2.6948,0.5000,-5.1000
8,6/30/2010,-3.7142,0.2800,0.0233,-2.6668,-4.0227,-4.2566,-3.4787,2.8305,0.5000,-5.3600
9,7/30/2010,3.9589,0.5500,0.0458,4.1103,3.7351,4.7640,2.3311,2.7326,0.6600,3.7100


In [4]:
dt = df.iloc[2:,1:].to_numpy()
dt = dt.astype(float)
dt = dt * (1/100)

In [5]:
#2
#averages
means = np.mean(dt, axis=0)
rm_mean, rf_mean, rp_mean = means[0], means[2], means[-1]
stds = np.std(dt, axis=0)
std = [stds[0], stds[2], stds[-1]]
asys = ss.skew(dt, axis=0)
asy = [asys[:2], asys[-1]]
kurts = ss.kurtosis(dt, axis=0)
kurt = [kurts[:2], kurts[-1]]

In [6]:
#3 Regression OLS rp-rf par rapport a rm-rf avec constante
y = dt[:, -1] - dt[:, 2]
x = sm.add_constant(dt[:, 0] - dt[:, 1])
rp_reg = sm.OLS(y, x).fit()
beta = rp_reg.params[1]
cov_m_p = np.cov(dt[:, [0, -1]], rowvar=False)[0, 1]
beta_t = cov_m_p / (std[0] ** 2)
beta_t

0.7705131627366909

In [7]:
#4
rm_high = pd.DataFrame(np.argsort(dt[:, 0]))
df2 = df.iloc[2:]
ranks = df2.iloc[rm_high.iloc[:, 0]]
rm_rp = ranks.iloc[:, [0, 1, -1]]
worst_10, best_10 = rm_rp.iloc[0:10], rm_rp.iloc[-1:10:-1]

In [8]:
#6

b_x = dt[:, [2, 3, 4, 5, 6]]
pos_coeff = so.lsq_linear(b_x, y, bounds=[0, 1])
pos_coeff.x

array([1.00000000e+00, 1.88358732e-01, 3.87141068e-01, 1.18136094e-09,
       1.55037035e-01])

In [9]:
#optimiseur

def lsos(betas, x, y):
    pred_y = np.sum([x[:, i] * betas[i] for i in np.arange(x.shape[1])], axis=0)
    res = np.sum((pred_y - y)**2)
    return res

bnds = [(0, 1) for i in range(b_x.shape[1])]
cons = {"type": "eq", "fun": lambda betas: np.sum(betas) - 1}
guess = pos_coeff.x

sh_clean = so.minimize(fun = lsos, x0=guess, args = (b_x, y), constraints = cons) 
sh_clean.x
#ols_me = so.minimize(fun=lsos, x0=[1, 1], args = (b_x, y))

array([ 0.28029021,  0.22010474,  0.90710587, -0.35634276, -0.05115807])

In [10]:
def r_squared(betas, x, y):
    pred_y = np.sum([x[:, i] * betas[i] for i in np.arange(x.shape[1])], axis=0)
    rss = np.sum((y - pred_y) ** 2)
    avg_y = np.mean(y)
    tss = np.sum((y - avg_y) ** 2)
    r_2 = 1 - (rss/tss)
    return r_2

In [21]:
#6 Sharpe style analysis

#W/ short selling allowed

sh_clean = so.minimize(fun = lsos, x0=guess, args = (b_x, y), constraints = cons) 
clean_betas = sh_clean.x
#w/o short selling
sh_cst = so.minimize(fun = lsos, x0=guess, args = (b_x, y), bounds = bnds, constraints = cons)
cst_betas = sh_cst.x

np.around(cst_betas, 3), np.around(clean_betas, 3)
r_squared(clean_betas, b_x, y), r_squared(cst_betas, b_x, y)

(0.643022361897352, 0.6350660319380439)

In [12]:
#7

sharpe_ratio = (rp_mean - rf_mean) / std[-1]
trey_ratio = (rp_mean - rf_mean) / beta
sharpe_ratio, trey_ratio

(0.3391679985078331, 0.01422972715221282)

In [13]:
#8

div_ratio = np.corrcoef(dt[:, [0, -1]], rowvar=False)[0, 1] ** 2
div_ratio

0.6552035206907063

In [14]:
#9

rp_rf = dt[:, -1] - dt[:, 2]
rm_rf = dt[:, 0] - dt[:, 2]
smb = dt[:, 3] - dt[:, 4]
hml = dt[:, 5] - dt[:, 6]
rp_rf_m, rm_rf_m, smb_m, hml_m = np.mean(rp_rf), np.mean(rm_rf), np.mean(smb), np.mean(hml)
rp_rf_std, rm_rf_std, smb_std, hml_std = np.std(rp_rf), np.std(rm_rf), np.std(smb), np.mean(hml)

In [15]:
#10

rp_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     257.3
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           1.05e-32
Time:                        11:02:46   Log-Likelihood:                 341.27
No. Observations:                 133   AIC:                            -678.5
Df Residuals:                     131   BIC:                            -672.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0124      0.002      7.591      0.000       0.009       0.016
x1             0.7622      0.048     16.041      0.000       0.668       0.856
==============================================================================
Omnibus:                        0.638   Durbin-Watson:                   1.503
Prob(Omnibus):                  0.727   Jarque-Bera (JB):                0.646
Skew:                           0.162   Prob(JB):                        0.724
Kurtosis:                       2.895   Cond. No.                         29.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
#11

fam_x = sm.add_constant(np.vstack((rm_rf, smb, hml)).transpose())
fam_reg = sm.OLS(y, fam_x).fit()
fam_reg.summary(yname='Rp', xname=['\u03B1 Jensen', '\u03B2m', '\u03B2sml', '\u03B2hml'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rp   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.670
Method:                 Least Squares   F-statistic:                     90.28
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           1.54e-31
Time:                        11:02:46   Log-Likelihood:                 344.24
No. Observations:                 133   AIC:                            -680.5
Df Residuals:                     129   BIC:                            -668.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
α Jensen       0.0069      0.002      4.183      0.000       0.004       0.010
βm             0.6988      0.056     12.488      0.000       0.588       0.809
βsml           0.1357      0.056      2.429      0.017       0.025       0.246
βhml          -0.0967      0.054     -1.776      0.078      -0.204       0.011
==============================================================================
Omnibus:                        0.390   Durbin-Watson:                   1.522
Prob(Omnibus):                  0.823   Jarque-Bera (JB):                0.518
Skew:                           0.113   Prob(JB):                        0.772
Kurtosis:                       2.795   Cond. No.                         43.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
#12
tm_x = sm.add_constant(np.vstack((rm_rf, (rm_rf ** 2))).transpose())
timing_tm = sm.OLS(y, tm_x).fit()
timing_tm.summary('Rp', ['\u03B1', '\u03B2m', '\u03B4m'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rp   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     130.2
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           9.00e-32
Time:                        11:02:46   Log-Likelihood:                 342.15
No. Observations:                 133   AIC:                            -678.3
Df Residuals:                     130   BIC:                            -669.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
α              0.0051      0.002      2.843      0.005       0.002       0.009
βm             0.7924      0.049     16.009      0.000       0.694       0.890
δm             1.1547      0.545      2.120      0.036       0.077       2.232
==============================================================================
Omnibus:                        0.773   Durbin-Watson:                   1.594
Prob(Omnibus):                  0.679   Jarque-Bera (JB):                0.818
Skew:                           0.178   Prob(JB):                        0.664
Kurtosis:                       2.855   Cond. No.                         336.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
hm_x = sm.add_constant(np.vstack((rm_rf, np.where(rm_rf > 0, rm_rf, 0))).transpose())
hm_reg = sm.OLS(y, hm_x).fit()
hm_reg.summary('Rp', ['\u03B1', '\u03B2m', '\u03BBm'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rp   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     130.0
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           9.60e-32
Time:                        11:02:46   Log-Likelihood:                 342.08
No. Observations:                 133   AIC:                            -678.2
Df Residuals:                     130   BIC:                            -669.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
α              0.0030      0.002      1.232      0.220      -0.002       0.008
βm             0.6324      0.080      7.929      0.000       0.475       0.790
λm             0.2881      0.138      2.088      0.039       0.015       0.561
==============================================================================
Omnibus:                        0.517   Durbin-Watson:                   1.549
Prob(Omnibus):                  0.772   Jarque-Bera (JB):                0.513
Skew:                           0.146   Prob(JB):                        0.774
Kurtosis:                       2.912   Cond. No.                         94.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
#13 conditionnelle capm

condtm_x = sm.add_constant(np.vstack((rm_rf[1:], rm_rf[1:] * dt[:-1, -2], rm_rf[1:] * dt[:-1, -3])).transpose())
condtm_reg = sm.OLS(y[1:], condtm_x).fit()
condtm_reg.summary('Rp', ['\u03B1', '\u03B2m', '\u03B2dpm', '\u03B2tbm'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rp   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     106.0
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           1.52e-34
Time:                        11:02:46   Log-Likelihood:                 349.30
No. Observations:                 132   AIC:                            -690.6
Df Residuals:                     128   BIC:                            -679.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
α              0.0052      0.002      3.343      0.001       0.002       0.008
βm             0.5831      0.429      1.360      0.176      -0.265       1.431
βdpm         -38.8955      8.286     -4.694      0.000     -55.291     -22.500
βtbm          17.5436     13.207      1.328      0.186      -8.588      43.676
==============================================================================
Omnibus:                        1.809   Durbin-Watson:                   1.574
Prob(Omnibus):                  0.405   Jarque-Bera (JB):                1.590
Skew:                           0.269   Prob(JB):                        0.452
Kurtosis:                       3.017   Cond. No.                     8.77e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.77e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
#13 conditionnelle treynor-mazuy

condj_x = np.vstack((condtm_x.transpose(), tm_x[1:, -1])).transpose()
condj_reg = sm.OLS(y[1:], condj_x).fit()
condj_reg.summary('Rp', ['\u03B1', '\u03B2m', '\u03B2dpm', '\u03B2tbm', '\u03B4m'])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     Rp   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     80.08
Date:                Thu, 15 Apr 2021   Prob (F-statistic):           8.81e-34
Time:                        11:02:46   Log-Likelihood:                 350.00
No. Observations:                 132   AIC:                            -690.0
Df Residuals:                     127   BIC:                            -675.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
α              0.0060      0.002      3.536      0.001       0.003       0.009
βm             0.5417      0.429      1.261      0.210      -0.308       1.392
βdpm         -45.9016     10.252     -4.477      0.000     -66.188     -25.615
βtbm          20.4244     13.422      1.522      0.131      -6.135      46.984
δm            -0.7489      0.647     -1.158      0.249      -2.029       0.531
==============================================================================
Omnibus:                        1.749   Durbin-Watson:                   1.538
Prob(Omnibus):                  0.417   Jarque-Bera (JB):                1.470
Skew:                           0.257   Prob(JB):                        0.479
Kurtosis:                       3.064   Cond. No.                     8.87e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.87e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""